In [1]:
from sklearn.metrics import f1_score, precision_score, recall_score
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
import random
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
from torch.nn.parallel import DataParallel
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("/kaggle/input/emotions/text.csv")
df.head()

,Unnamed: 0,text,label
0,0,i just feel really helpless and heavy hearted,4
1,1,ive enjoyed being able to slouch about relax a...,0
2,2,i gave up my internship with the dmrg and am f...,4
3,3,i dont know i feel so lost,0
4,4,i am a kindergarten teacher and i am thoroughl...,4


In [3]:
class TextStratifiedData(Dataset):
    def __init__(self, df, length=None):
        if length is not None and length > df.shape[0]:
            raise ValueError("Length parameter cannot be greater than the size of the dataset.")
        self.length = length if length is not None else len(df)
        self.df = self.stratify(df)
 
    def stratify(self, df):
        min_count = df['label'].value_counts().min()
        df = df.groupby('label').apply(lambda x: x.sample(min_count)).reset_index(drop=True)
        return df.sample(self.length)

    def len(self):
        return self.df.shape[0]

    def get_item(self, idx):
        return self.df.iloc[idx, :]
    
    def get_all(self):
        return self.df
    
    
df = TextStratifiedData(df,25000).get_all()

df.label.value_counts()

/tmp/ipykernel_34/3363077029.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('label').apply(lambda x: x.sample(min_count)).reset_index(drop=True)


label
4    4207
5    4196
2    4182
3    4181
0    4141
1    4093
Name: count, dtype: int64

In [4]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")

train_texts, val_texts, train_labels, val_labels = train_test_split(df["text"], df["label"], test_size=0.2, random_state=42)

train_texts = train_texts.tolist()
val_texts = val_texts.tolist()
train_labels = np.array(train_labels)
val_labels = np.array(val_labels)

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128, return_tensors='pt')
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128, return_tensors='pt')

train_dataset = torch.utils.data.TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], torch.tensor(train_labels,dtype=torch.float64))
val_dataset = torch.utils.data.TensorDataset(val_encodings['input_ids'], val_encodings['attention_mask'], torch.tensor(val_labels,dtype=torch.float64))

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [5]:
class EmotionClassifierWithConv(nn.Module):
    def __init__(self, transformer_model, num_classes, kernel_size=3, num_filters=256):
        super(EmotionClassifierWithConv, self).__init__()
        self.transformer = transformer_model
        self.conv = nn.Conv1d(in_channels=768, out_channels=num_filters, kernel_size=kernel_size, padding=1)  # Adjust padding
        self.fc = nn.Linear(num_filters, num_classes)
    
    def forward(self, input_ids, attention_mask):
        output = self.transformer(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = output.pooler_output
        pooled_output = pooled_output.unsqueeze(2)
        
        conv_out = F.relu(self.conv(pooled_output))
        pooled_conv_out, _ = torch.max(conv_out, dim=2)  
        logits = self.fc(pooled_conv_out)
        return logits


In [6]:
num_classes =6  
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device_ids = [0, 1]  


model = EmotionClassifierWithConv(model, num_classes)
model = nn.DataParallel(model, device_ids=device_ids)
model = model.to(device)


optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
criterion = nn.CrossEntropyLoss()


num_epochs=3
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    total_correct = 0
    
    for batch in train_dataloader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
       
        labels = labels.to(device).long()
        outputs = outputs.float()
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total_correct += (predicted == labels).sum().item()
    
    train_loss = total_loss / len(train_dataloader)
    train_accuracy = total_correct / len(train_dataset)
    
    # Validation loop
    model.eval()
    total_val_loss = 0.0
    total_val_correct = 0
    val_predicted = []
    val_labels = []
    
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            outputs = model(input_ids, attention_mask)
            labels = labels.to(device).long()
            
            outputs = outputs.float()
            loss = criterion(outputs, labels)

            total_val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total_val_correct += (predicted == labels).sum().item()
            val_predicted.extend(predicted.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())
    

In [7]:
from sklearn.metrics import classification_report


val_predicted = np.array(val_predicted)
val_labels = np.array(val_labels)


report = classification_report(val_labels, val_predicted, target_names=[f'Class {i}' for i in range(num_classes)])

print("Classification Report:")
print(report)

Classification Report:
              precision    recall  f1-score   support

     Class 0       0.96      0.94      0.95       797
     Class 1       0.97      0.90      0.93       821
     Class 2       0.93      0.97      0.95       824
     Class 3       0.93      0.96      0.95       836
     Class 4       0.95      0.87      0.91       863
     Class 5       0.91      1.00      0.95       859

    accuracy                           0.94      5000
   macro avg       0.94      0.94      0.94      5000
weighted avg       0.94      0.94      0.94      5000



In [8]:
import zipfile


torch.save(model.state_dict(), 'model_weights40k.pth')


with zipfile.ZipFile('/kaggle/working/model_weights.zip', 'w') as zip_f:
    zip_f.write('model_weights40k.pth')

from IPython.display import FileLink
FileLink(r'model_weights.zip')

/kaggle/working/model_weights.zip